In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools
import cv2
from PIL import Image

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from skimage import io
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from keras.models import load_model
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
%matplotlib inline

In [ ]:
emotion_classifier = load_model('models.h5')
# df = pd.read_csv('Dataset.csv',encoding='utf-8')

NameError: name 'load_model' is not defined

In [ ]:
spotify_df = df.copy()

In [ ]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [ ]:
ohe_cols = 'popularity'

In [ ]:
spotify_df['bucket_popularity'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [ ]:
spotify_df['consolidates_genre_lists_upd'] = spotify_df['consolidates_genre_lists'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [ ]:
spotify_df.head()

,Mood,Artists,Name,popularity,consolidates_genre_lists,bucket_popularity,consolidates_genre_lists_upd
0,Happy,John Legend,All of Me,85,"['pop', 'r&b', 'soul']",17,"[pop, r&b, soul]"
1,Sad,Adele,Someone Like You,92,"['pop', 'soul', 'piano']",18,"[pop, soul, piano]"
2,Angry,Linkin Park,Numb,78,"['rock', 'alternative']",15,"[rock, alternative]"
3,Happy,Pharrell Williams,Happy,88,"['pop', 'funk', 'r&b']",17,"[pop, funk, r&b]"
4,Sad,Coldplay,Fix You,70,"['rock', 'alternative']",14,"[rock, alternative]"


In [ ]:
client_id = '1eb05bcd5fdd4607ae338444b787b426'
client_secret = '1a8fc32470d34264a2a6df77ef3716e1'

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
token = util.prompt_for_user_token(client_id=client_id, client_secret=client_secret, redirect_uri='http://localhost:3000')

SpotifyOauthError: error: invalid_client, error_description: Invalid client

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']
    
id_name
print(id_name)
#first_key= next(iter(id_name))
#first_value= id_name[first_key]
first_key='My Playlist #2'
first_value= id_name[first_key]
playlist_id= first_value
print(playlist_id)

In [ ]:
def ChooseDataset(x):
    if x == "Disgust":
        return spotify_df[spotify_df['Mood'].isin(['Sad', 'Calm'])]
    if x == "Angry":
        return spotify_df[spotify_df['Mood'].isin(['Calm'])]
    if x == "Fear":
        return spotify_df[spotify_df['Mood'].isin(['Calm'])]
    if x == "Happy":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Happy'])]
    if x == "Sad":
        return spotify_df[spotify_df['Mood'].isin(['Sad', 'Calm'])]
    if x == "Surprise":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Happy', 'Calm'])]
    return spotify_df

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from io import BytesIO
from PIL import Image

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id,
                                                                             client_secret=client_secret))

In [ ]:
def get_user_playlist(user_id, playlist_id):
    results = spotify.user_playlist(user_id, playlist_id)
    tracks = results['tracks']['items']
    playlist = []
    for track in tracks:
        playlist.append(track['track']['name'])
    return playlist

In [ ]:
def preprocess_data(dataset, playlist):
    # Remove any missing values in dataset
    dataset = dataset.dropna()

    # Preprocess dataset and playlist by combining relevant text fields
    dataset['text'] = dataset['artists'] + ' ' + dataset['name']
    playlist_text = ' '.join(playlist)
    dataset = pd.concat([dataset, pd.DataFrame({'text': [playlist_text]})], ignore_index=True)
    return dataset

In [ ]:
def recommend_songs(dataset, playlist_text, top_n=30):
    # Vectorize text data using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(dataset['text'])

    # Calculate cosine similarity between playlist and dataset songs
    playlist_vector = tfidf_matrix[-1]  # Last row corresponds to user playlist
    similarities = cosine_similarity(playlist_vector, tfidf_matrix[:-1])  # Exclude user playlist from similarity calculation

    # Get indices of top similar songs
    top_indices = similarities.argsort()[0][::-1][:top_n]

    # Return recommended songs
    recommended_songs = dataset.iloc[top_indices]
    return recommended_songs

In [ ]:
def visualize_songs(recommended_songs):
    for _, song in recommended_songs.iterrows():
        track_info = spotify.search(q='track:"{}" artist:"{}"'.format(song['name'], song['artists']), type='track')
        if track_info['tracks']['items']:
            image_url = track_info['tracks']['items'][0]['album']['images'][0]['url']
            song_url = track_info['tracks']['items'][0]['external_urls']['spotify']
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content))
            plt.imshow(image)
            plt.title(song['name'] + ' - ' + song['artists'])
            plt.axis('off')
            plt.show()
            print("Song URL: ",song_url)

In [ ]:
from keras.models import load_model
from time import sleep
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
classifier =load_model(r'model.h5')

emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)



while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),10)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)



        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

finalEmotion=""
print("Prediction array: ",prediction)
maxIndex=0;
max=0;
for i in range(len(prediction)):
    if prediction[i]>max:
        max=prediction[i]
        maxIndex=i

if(maxIndex==0):
    finalEmotion="Angry"
elif(maxIndex==1):
    finalEmotion="Disgust"
elif(maxIndex==2):
    finalEmotion="Fear"
elif(maxIndex==3):
    finalEmotion="Happy"
elif(maxIndex==4):
    finalEmotion="Neutral"
elif(maxIndex==5):
    finalEmotion="Sad"
elif(maxIndex==6):
    finalEmotion="Suprise"
print("Predicted Emotion: ",finalEmotion)
print("Recommended songs are as under: ")
user_id = 'x75tchtjdlipeyza91spkk5i8'
dataset = ChooseDataset(finalEmotion)
user_playlist = get_user_playlist(user_id, playlist_id)
preprocessed_data = preprocess_data(dataset, user_playlist)
recommended_songs = recommend_songs(preprocessed_data, user_playlist)
print(recommended_songs[['name']])
visualize_songs(recommended_songs)